In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:,.2f}'.format)

In [5]:
ticker = "BP.L"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [6]:
data = data['2022-07-10':]
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-07-11 00:00:00+01:00,347.624751,353.057798,344.885391,351.185913,32272439,0.0,0.0
2022-07-12 00:00:00+01:00,351.779434,352.464274,341.050275,344.291840,36790825,0.0,0.0
2022-07-13 00:00:00+01:00,343.561328,350.090129,339.954493,344.428802,38576591,0.0,0.0
2022-07-14 00:00:00+01:00,345.067960,347.350760,327.992637,332.329956,62414862,0.0,0.0
2022-07-15 00:00:00+01:00,335.434529,344.383091,333.334337,340.684967,39243191,0.0,0.0
...,...,...,...,...,...,...,...
2024-07-15 00:00:00+01:00,447.450012,456.250000,444.299988,449.399994,22978638,0.0,0.0
2024-07-16 00:00:00+01:00,448.299988,452.100006,443.000000,446.600006,23748366,0.0,0.0
2024-07-17 00:00:00+01:00,446.799988,454.463989,445.859985,452.700012,21130713,0.0,0.0


In [7]:
strategy = SmaCross
strategy.n1 = 20
strategy.n2 = 40
bt_sma = Backtest(data, SmaCross, cash=10_000, commission=0.004, trade_on_close=False)
stats = bt_sma.run()
stats

Start                     2022-07-11 00:00...
End                       2024-07-19 00:00...
Duration                    739 days 00:00:00
Exposure Time [%]                   86.862745
Equity Final [$]                   9362.46253
Equity Peak [$]                  10898.478602
Return [%]                          -6.375375
Buy & Hold Return [%]               30.215926
Return (Ann.) [%]                   -3.202679
Volatility (Ann.) [%]               21.758911
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.407954
Avg. Drawdown [%]                   -8.873815
Max. Drawdown Duration      519 days 00:00:00
Avg. Drawdown Duration      107 days 00:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       7.578345
Worst Trade [%]                     -9.682627
Avg. Trade [%]                    

In [9]:
bt_sma.plot()

Row(id='2182', ...)

In [34]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 2.24 s, sys: 73.1 ms, total: 2.32 s
Wall time: 2.27 s


Start                     2023-07-10 00:00...
End                       2024-07-17 00:00...
Duration                    373 days 00:00:00
Exposure Time [%]                   71.657754
Equity Final [$]                 13678.588331
Equity Peak [$]                  18796.377434
Return [%]                          36.785883
Buy & Hold Return [%]               80.531833
Return (Ann.) [%]                   35.758665
Volatility (Ann.) [%]               62.936102
Sharpe Ratio                         0.568174
Sortino Ratio                        1.205628
Calmar Ratio                         1.270801
Max. Drawdown [%]                  -28.138681
Avg. Drawdown [%]                   -6.191831
Max. Drawdown Duration       64 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      41.061413
Worst Trade [%]                    -12.548966
Avg. Trade [%]                    

In [35]:
stats._strategy

<Strategy SmaCross(n1=25,n2=50)>

In [36]:
bt_sma.plot()

Row(id='5601', ...)

In [46]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

CPU times: user 582 ms, sys: 18.6 ms, total: 600 ms
Wall time: 588 ms


Start                     2023-07-10 00:00...
End                       2024-07-17 00:00...
Duration                    373 days 00:00:00
Exposure Time [%]                   16.042781
Equity Final [$]                  1462.974438
Equity Peak [$]                       10000.0
Return [%]                         -85.370256
Buy & Hold Return [%]               80.531833
Return (Ann.) [%]                  -84.677677
Volatility (Ann.) [%]                3.859311
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.370256
Avg. Drawdown [%]                  -85.370256
Max. Drawdown Duration      368 days 00:00:00
Avg. Drawdown Duration      368 days 00:00:00
# Trades                                   58
Win Rate [%]                         3.448276
Best Trade [%]                       2.959956
Worst Trade [%]                      -9.04646
Avg. Trade [%]                    

In [47]:
bt_momentum.plot()

Row(id='7361', ...)

In [49]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=True)
stats = bt_momentum.run()
stats

Start                     2023-07-10 00:00...
End                       2024-07-17 00:00...
Duration                    373 days 00:00:00
Exposure Time [%]                   16.042781
Equity Final [$]                  1466.796162
Equity Peak [$]                       10000.0
Return [%]                         -85.332038
Buy & Hold Return [%]               80.531833
Return (Ann.) [%]                  -84.638615
Volatility (Ann.) [%]                3.854725
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.332038
Avg. Drawdown [%]                  -85.332038
Max. Drawdown Duration      369 days 00:00:00
Avg. Drawdown Duration      369 days 00:00:00
# Trades                                   58
Win Rate [%]                         3.448276
Best Trade [%]                       2.797872
Worst Trade [%]                     -9.037462
Avg. Trade [%]                    